<div style="text-align: center">
  <a href="https://www.aicrowd.com/challenges/rl-taxi"><img alt="AIcrowd" src="https://images.aicrowd.com/raw_images/challenges/banner_file/759/d9540ebbd506b68a5ff2.jpg"></a>
</div>

# What is the notebook about?

## Problem - DP Algorithm
This problem deals with a taxi driver with multiple actions in different cities. The tasks you have to do are:
- Implement DP Algorithm to find the optimal sequence for the taxi driver
- Find optimal policies for sequences of varying lengths
- Explain a variation on the policy

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.


- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_DATASET_PATH` | Path to the file containing test data. This should be an absolute path.
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

# Setup AIcrowd Utilities 🛠

We use this to bundle the files for submission and create a submission on AIcrowd. Do not edit this block.

In [ ]:
!pip install -U git+https://gitlab.aicrowd.com/aicrowd/aicrowd-cli.git@notebook-submission-v2 > /dev/null 

  Running command git clone -q https://gitlab.aicrowd.com/aicrowd/aicrowd-cli.git /tmp/pip-req-build-mse0o8hu
  Running command git checkout -b notebook-submission-v2 --track origin/notebook-submission-v2
  Switched to a new branch 'notebook-submission-v2'
  Branch 'notebook-submission-v2' set up to track remote branch 'notebook-submission-v2' from 'origin'.


# AIcrowd Runtime Configuration 🧷

Define configuration parameters. Please include any files needed for the notebook to run under `ASSETS_DIR`. We will copy the contents of this directory to your final submission file 🙂

In [ ]:
import os

AICROWD_DATASET_PATH = os.getenv("DATASET_PATH", os.getcwd()+"/40746340-4151-4921-8496-be10b3f8f5cf_hw2_q1.zip")
AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
API_KEY = "6b42b562b9eb39fe9d07fb070f706ef5" #Get your API key from https://www.aicrowd.com/participants/me

# Download dataset files 📲

In [ ]:
!aicrowd login --api-key $API_KEY
!aicrowd dataset download -c rl-taxi

API Key valid
Saved API Key successfully!
40746340-4151-4921-8496-be10b3f8f5cf_hw2_q1.zip: 100% 3.03k/3.03k [00:00<00:00, 157kB/s]


In [ ]:
!unzip -q $AICROWD_DATASET_PATH

In [ ]:
DATASET_DIR = 'hw2_q1/'
!mkdir {DATASET_DIR}results/

mkdir: cannot create directory ‘hw2_q1/results/’: File exists


# Install packages 🗃

Please add all pacakage installations in this section

# Import packages 💻

In [ ]:
import numpy as np
import os
# ADD ANY IMPORTS YOU WANT HERE

In [ ]:
import numpy as np

class TaxiEnv_HW2:
    def __init__(self, states, actions, probabilities, rewards):
        self.possible_states = states
        self._possible_actions = {st: ac for st, ac in zip(states, actions)}
        self._ride_probabilities = {st: pr for st, pr in zip(states, probabilities)}
        self._ride_rewards = {st: rw for st, rw in zip(states, rewards)}
        self._verify()

    def _check_state(self, state):
        assert state in self.possible_states, "State %s is not a valid state" % state

    def _verify(self):
        """ 
        Verify that data conditions are met:
        Number of actions matches shape of next state and actions
        Every probability distribution adds up to 1 
        """
        ns = len(self.possible_states)
        for state in self.possible_states:
            ac = self._possible_actions[state]
            na = len(ac)

            rp = self._ride_probabilities[state]
            assert np.all(rp.shape == (na, ns)), "Probabilities shape mismatch"
        
            rr = self._ride_rewards[state]
            assert np.all(rr.shape == (na, ns)), "Rewards shape mismatch"

            assert np.allclose(rp.sum(axis=1), 1), "Probabilities don't add up to 1"

    def possible_actions(self, state):
        """ Return all possible actions from a given state """
        self._check_state(state)
        return self._possible_actions[state]

    def ride_probabilities(self, state, action):
        """ 
        Returns all possible ride probabilities from a state for a given action
        For every action a list with the returned with values in the same order as self.possible_states
        """
        actions = self.possible_actions(state)
        ac_idx = actions.index(action)
        return self._ride_probabilities[state][ac_idx]

    def ride_rewards(self, state, action):
        actions = self.possible_actions(state)
        ac_idx = actions.index(action)
        return self._ride_rewards[state][ac_idx]

# Examples of using the environment functions

In [ ]:
def check_taxienv():
    # These are the values as used in the pdf, but they may be changed during submission, so do not hardcode anything

    states = ['A', 'B', 'C']

    actions = [['1','2','3'], ['1','2'], ['1','2','3']]

    probs = [np.array([[1/2,  1/4,  1/4],
                    [1/16, 3/4,  3/16],
                    [1/4,  1/8,  5/8]]),

            np.array([[1/2,   0,     1/2],
                    [1/16,  7/8,  1/16]]),

            np.array([[1/4,  1/4,  1/2],
                    [1/8,  3/4,  1/8],
                    [3/4,  1/16, 3/16]]),]

    rewards = [np.array([[10,  4,  8],
                        [ 8,  2,  4],
                        [ 4,  6,  4]]),
    
            np.array([[14,  0, 18],
                        [ 8, 16,  8]]),
    
            np.array([[10,  2,  8],
                        [6,   4,  2],
                        [4,   0,  8]]),]


    env = TaxiEnv_HW2(states, actions, probs, rewards)
    print("All possible states", env.possible_states)
    print("All possible actions from state B", env.possible_actions('B'))
    print("Ride probabilities from state A with action 2", env.ride_probabilities('A', '2'))
    print("Ride rewards from state C with action 3", env.ride_rewards('C', '3'))

check_taxienv()

All possible states ['A', 'B', 'C']
All possible actions from state B ['1', '2']
Ride probabilities from state A with action 2 [0.0625 0.75   0.1875]
Ride rewards from state C with action 3 [4 0 8]


# Task 1 - DP Algorithm implementation
Implement your DP algorithm that takes the starting state and sequence length
and return the expected reward for the policy

In [ ]:
def dp_solve(taxienv):
    ## Implement the DP algorithm for the taxienv
    states = taxienv.possible_states
    values = {s: 0 for s in states}
    policy = {s: '0' for s in states}
    all_values = [] # Append the "values" dictionary to this after each update
    all_policies = [] # Append the "policy" dictionary to this after each update
    
    # ADD YOUR CODE BELOW - DO NOT EDIT ABOVE THIS LINE
    nextJ = {s: 0 for s in states}
    for k in range(9,-1,-1):
      values = {s: 0 for s in states}
      policy = {s: '0' for s in states}
      for state in states:
        J = []
        for action in taxienv.possible_actions(state):
          vectora = taxienv.ride_probabilities(state, action)
          vectorb = taxienv.ride_rewards(state, action) + np.array(list(nextJ.values()))
          dot_product = 0
          for i in range(vectora.shape[0]):
            dot_product += (vectora[i]*vectorb[i])
          J.append(dot_product)
        values[state] = max(J)
        policy[state] = str(taxienv.possible_actions(state)[J.index(max(J))])
      nextJ = values.copy()
      all_values.append(values)
      all_policies.append(policy)
    # Note: 
    #   The sequence length is always N=10
    #   Append your policy and values dictionaries in the same order you update in the loop 

    # DO NOT EDIT BELOW THIS LINE
    results = {"Expected Reward": all_values, "Polcies": all_policies}
    return results

## Here is an example of what the "results" output from value_iter function should look like

Ofcourse, it won't be all zeros
``` python 
{'Expected Reward': [{'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0},
  {'A': 0, 'B': 0, 'C': 0}],
 'Polcies': [{'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'},
  {'A': '0', 'B': '0', 'C': '0'}]}

  ```

In [ ]:
if not os.path.exists(AICROWD_RESULTS_DIR):
  os.mkdir(AICROWD_RESULTS_DIR)
if not os.path.exists(DATASET_DIR+'/inputs'):
  os.mkdir(DATASET_DIR+'/inputs')

In [ ]:
# DO NOT EDIT THIS CELL, DURING EVALUATION THE DATASET DIR WILL CHANGE
input_dir = os.path.join(DATASET_DIR, 'inputs')
results = {}
for params_file in os.listdir(input_dir):
  kwargs = np.load(os.path.join(input_dir, params_file), allow_pickle=True).item()

  env = TaxiEnv_HW2(**kwargs)

  results = dp_solve(env)
  print(results)
  idx = params_file.split('_')[-1][:-4]
  np.save(os.path.join(AICROWD_RESULTS_DIR, 'results_' + idx), results)

{'Expected Reward': [{'A': 8.0, 'B': 16.0, 'C': 7.0}, {'A': 17.75, 'B': 29.9375, 'C': 17.875}, {'A': 29.6640625, 'B': 43.421875, 'C': 30.90625}, {'A': 42.96533203125, 'B': 56.77978515625, 'C': 44.1376953125}, {'A': 56.295989990234375, 'B': 70.12625122070312, 'C': 57.47271728515625}, {'A': 69.63932228088379, 'B': 83.47101402282715, 'C': 70.81577682495117}, {'A': 82.98367631435394, 'B': 96.81558096408844, 'C': 84.16014790534973}, {'A': 96.32819322496653, 'B': 110.16012235730886, 'C': 97.50466375052929}, {'A': 109.6727282977663, 'B': 123.50466062361374, 'C': 110.84919888991863}, {'A': 123.01726577818044, 'B': 136.84919849489233, 'C': 124.19373636617092}], 'Polcies': [{'A': '1', 'B': '1', 'C': '1'}, {'A': '1', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}]}


In [ ]:
## Modify this code to show the results for the policy and expected rewards properly
print(results)

{'Expected Reward': [{'A': 8.0, 'B': 16.0, 'C': 7.0}, {'A': 17.75, 'B': 29.9375, 'C': 17.875}, {'A': 29.6640625, 'B': 43.421875, 'C': 30.90625}, {'A': 42.96533203125, 'B': 56.77978515625, 'C': 44.1376953125}, {'A': 56.295989990234375, 'B': 70.12625122070312, 'C': 57.47271728515625}, {'A': 69.63932228088379, 'B': 83.47101402282715, 'C': 70.81577682495117}, {'A': 82.98367631435394, 'B': 96.81558096408844, 'C': 84.16014790534973}, {'A': 96.32819322496653, 'B': 110.16012235730886, 'C': 97.50466375052929}, {'A': 109.6727282977663, 'B': 123.50466062361374, 'C': 110.84919888991863}, {'A': 123.01726577818044, 'B': 136.84919849489233, 'C': 124.19373636617092}], 'Polcies': [{'A': '1', 'B': '1', 'C': '1'}, {'A': '1', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}, {'A': '2', 'B': '2', 'C': '2'}]}


# Task 2 - Tabulate the optimal policy & optimal value for each state in each round for N=10

Modify this cell and add your answer

# Question -  Consider a policy that always forces the driver to go to the nearest taxi stand, irrespective of the state. Is it optimal? Justify your answer.



Modify this cell and add your answer

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [ ]:
!DATASET_PATH=$AICROWD_DATASET_PATH aicrowd notebook submit -c rl-taxi -a assets

Using notebook: /content/Copy%20of%20Copy%20of%20IITM_RL_DP_ASSIGNMENT_v1 for submission...
Mounting Google Drive 💾
Your Google Drive will be mounted to access the colab notebook
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7bzxwc7ArJnxVqrq_fi4fAW_4jo7DRUrQLGmGUJYkS7W0X3U3zWB0
Mounted at /content/drive
Scrubbing API keys from the notebook...
Validating the submission...
Executing install.ipynb...
[NbConvertA